# Procesamiento de datos
Fecha: 2 Marzo 2022 

## Indice
1. Funciones y funciones lambda <br>
1.1. Interludio 1: Repaso de bucles for <br>
1.2. Interludio 2: Repaso de comprensión de listas <br>
2. Método apply <br>
3. **Combinación de Dataframes <br>
3.1 Método concat <br>
3.2 Método append <br>
3.3 Método merge <br>
3.4 Método join **<br>
4. **Reorganizacion de dataframes <br>
4.1 Método pivot <br>
4.2 Métodos stack / unstack** <br>

Antes de empezar, importamos los paquetes que vamos a utilizar:

In [ ]:
import pandas as pd

# Combinación de dataframes
Hay ocasiones en los que es necesario usar datos de diferentes dataframes, listas u otros contenedores de datos. Para hacer las operaciones de unir diferentes contenedores de datos, pandas tiene funciones como concat(), append(), merge() and join(). 


## Concat()
Cuando se concatenan series, el objeto devuelto es una serie. Cuando se concatenan varios objetos, incluyendo al menos un dataframe, el objeto resultante es un dataframe.

**Ejemplo**: concatenar dos series s1 y s2.

In [ ]:
# Ejemplo con series
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])

print('serie 1: ')
print(s1)

print('serie 2: ')
print(s2)

In [ ]:
print('series concatenadas: ')
pd.concat([s1, s2])

Si nos fijamos, se han mantenido los indices que traian las series. Si queremos un nuevo indice:

**Ejemplo**: concatenar dos series s1 y s2 ignorando el indice de entrada.

In [ ]:
pd.concat([s1, s2], ignore_index=True)

Esto tambien se puede hacer de forma jerarquica con keys:

**Ejemplo**: Concatenar dos series s1 y s2 especificando keys de origen.

In [ ]:
pd.concat([s1, s2], keys=['s1', 's2'])

Ahora vamos a probar con data frames. Para ello vamos a crear tres dataframes diferentes con los que jugar:

In [ ]:
df1 = pd.DataFrame([[1,2,3, 4], [5,6,7,8], [9,10,11,12]], columns =['A', 'B', 'C', 'D'])
df1

In [ ]:
df2 = pd.DataFrame([[13,14,15,16], [17,18,19,20]], columns =['A', 'B', 'C', 'D'])
df2

In [ ]:
df3 = pd.DataFrame([[21,22], [23,24], [25,26]], columns =['F', 'G'])
df3

**Ejemplo:** Si queremos concatenar dos DataFrames de forma que el segundo sean filas nuevas del primero, podemos aplicar el método concat() dejando el valor de axis por defecto (axis=0).
Nota: Recuerda que si queremos nuevos indices para las filas anadidas tenemos que escribir ignore_index=True

In [ ]:
#pd.concat([df1,df2]) # Si no nos importa que los indices queden como estaban.

pd.concat([df1,df2], ignore_index=True) # Si queremos que los indices se reenumeren.

**Ejemplo:** Si queremos que el segundo DataFrame aporte columnas nuevas al primero, debemos llamar al método concat con la opción axis=1.

In [ ]:
pd.concat([df1,df3], axis = 1)

Pero, que pasa si los dataframes que queremos concatenar no tienen la misma forma?

In [ ]:
pd.concat([df2,df3], axis = 1)

En este caso, nos aparecen valores NaN (Not-a-Number) rellenando los elementos vacios.

## Append()
Pandas.append() sirve para unir filas de un objeto a las filas del objeto de llamada. 

**Ejemplo:** Agregar las filas de df2 a df1.

In [ ]:
df1.append(df2, ignore_index=True)

Append sería equivalente a concat con axis = 0. La diferencia es que append agrega los elementos de forma iterativa, mientras que concat lo hace en una sola operación. Para pocas filas, no hay diferencia, pero para muchas operaciones, el tiempo puede ser un factor importante.

## Merge()

Merge() puede unir series o dataframes usando tanto columnas como indices. Sería equivalente a join en bases de datos relacionales como SQL.

**Ejemplo**: Usar merge() para unir los siguientes dataframes df1 y df2 usando 'id' como key.

In [ ]:
# dataframe 1
df1 = pd.DataFrame([['1','Estudio', '2'], ['2', 'Nave industrial', '1'],['3','Casa de campo', '5'], ['4','Estudio', '1']],
                    columns=['id', 'Inmueble', 'Habitaciones'])


df1

In [ ]:
# dataframe 2
df2 = pd.DataFrame([['1','32', '2'], ['2', '130', '1'],['3','70', '5'],['5','65', '4'], ['6','55', '2']], 
                   columns=['id', 'Metros cuadrados', 'Habitaciones'])

df2

In [ ]:
pd.merge(df1, df2, on='id')

Como podemos observar, la columna Habitaciones se ha duplicado. Como es igual para los dos, también podríamos usarla para unir los dataframes, de forma que no aparezca duplicada.

In [ ]:
pd.merge(df1, df2, on=['id', 'Habitaciones'])

Si no especifico que columnas usar, escogerá automáticamente las columnas con nombres iguales:

In [ ]:
pd.merge(df1, df2)

En general, será recomendable especificar que key usar.

Si los nombres de las columnas son diferentes en cada dataframe, se pueden especificar al hacer el merge. Por ejemplo, si las columnas id y habitaciones no se llaman igual:

In [ ]:
# dataframe 1
df3 = pd.DataFrame([['1','Estudio', '2'], ['2', 'Nave industrial', '1'],['3','Casa de campo', '5'], ['4','Estudio', '1']],
                    columns=['id number', 'Inmueble', 'Num de Habitaciones'])


# dataframe 2
df4 = pd.DataFrame([['1','32', '2'], ['2', '130', '1'],['3','70', '5'],['5','65', '4'], ['6','55', '2']], 
                   columns=['id', 'Metros cuadrados', 'Habitaciones'])


In [ ]:
pd.merge(df3,df4, left_on = 'id number', right_on = 'id')

Hasta aquí, estamos viendo que en nuestro dataframe final solo aparecen las filas que estan en ambos dataframes. Esto se llama "inner join". 

In [ ]:
pd.merge(df1, df2, how = 'inner')

También tenemos opción de que salgan todos los elementos, dejando valores NaN en los huecos ("Outer join").

In [ ]:
pd.merge(df1, df2, how = 'outer')

O bien, centrarnos en los elementos de un dataframe u otro.

In [ ]:
pd.merge(df1, df2, how = 'left')

In [ ]:
pd.merge(df1, df2, how = 'right')

## Join()

Una alternativa a Merge() es usar Join(). Join() une columnas de diferentes dataframes. Para ello, podemos usar indices o columnas como keys.
El resultado de la función join() es un dataframe que contiene columnas de ambos dataframes.

**Ejemplo**: Usar join() para unir df1 y df2.

In [ ]:
df1.join(df2, lsuffix='_caller', rsuffix='_df2')

Como vemos, join trabaja sobre df1, y agrega de df2 la información que complemente sus filas.

**Ejercicio**: Como lo haríamos al reves, si lo que queremos complementar son las filas en df2?

## Diferencias entre concat(), append(), merge() y join()  

Las cuatro funciones usadas en pandas para unir series o dataframes operan de una forma muy parecida. Estas son las principales diferencias:

Append() es un caso específico de concat(). Tiene por tanto una funcionalidad limitada; solo puede usarse para filas y es mas lento.

Concat() puede ser usado para unir dataframes a traves de columnas o filas. Es supuesta como la opción mas funcional (mas rápida).

Merge() puede usarse como un join similar a SQL. Para usarla se puede usar una columna igual en los dos dataframes.

La diferencia entre join() y merge() es que join() usa el indice del lado izquierdo, mientras que merge() usa un nuevo indices para mostrar los resultados.


# El método pivot

Es común representar conjuntos de datos en dos tipos de formato: el formato long y el formato wide. Ambos formatos contienen la misma información, pero en ocasiones nos será conveniente pasar de uno a otro para analizar los datos o para almacenarlos. El métod o pivot nos servirá para pasar de long a wide.

Podremos trabajar con varias columnas de valores, dando lugar a un multiindex . Como contrapartida, este método no soporta agregación de datos. Para ello, deberemos usar pivot_table.

Veamos un **Ejemplo**: El siguiente dataframe esta escrito en formato long

In [ ]:
print('FORMATO LONG:')
df5 = pd.DataFrame([[100, 200, 'Q1', 'Madrid'], [90, 250, 'Q2', 'Madrid'],[110, 190, 'Q3', 'Madrid'],[120, 220, 'Q4', 'Madrid'],[70, 100, 'Q1', 'Toledo'], [55, 150, 'Q2', 'Toledo'],[65, 110, 'Q3', 'Toledo'],[70, 120, 'Q4', 'Toledo']], columns = ['gastos', 'ventas', 'trimestre', 'sucursal'])
df5

Este formato se puede cambiar a wide:

In [ ]:
print('FORMATO WIDE:')
df6 = df5.pivot(index = 'trimestre', columns = 'sucursal', values = ['gastos', 'ventas'])
df6

Como nota adicional, podemos cambiar los niveles del multiindex y reordenarlo. Con ello podremos facilitar la interpretación de algunos conjuntos de datos.

In [ ]:
df6.columns = df6.columns.swaplevel(0,1)
df6

In [ ]:
df6.sort_index(axis=1, level=0, inplace=True)
df6

## Stack / Unstack
El método pivot es un caso particular de una operación sobre DataFrames llamada ' stack '. Aplicar stack sobre un DataFrame significa mover la capa más interna de las columnas hacia la capa más interna del índice. La operación contraria es ' unstack '. Utilizaremos 'unstack' para pasar de formato wide a formato long, de la misma forma que usábamos pivot para pasar de long a wide.

**Ejemplo**: Vamos a usar stack y unstack con df6.

In [ ]:
df6

In [ ]:
df6.stack()

In [ ]:
df6.stack().unstack()

In [ ]:
df6.stack().unstack(level=0)